# Leitura e Pré-processamento dos Registros da Base MIT-BIH

Este notebook realiza a leitura, pré-processamento e análise dos registros da base MIT-BIH Arrhythmia Database.

## Etapas do processo:
1. Carregar e verificar os registros disponíveis.
2. Processar os registros com o canal MLII (canal padrão de ECG).
3. Extrair os batimentos cardíacos e suas classes.
4. Salvar os dados extraídos em um arquivo CSV.

## 1. Importação de Bibliotecas

- `wfdb`: Para ler os registros de ECG da base MIT-BIH.
- `os`: Para manipulação de arquivos e diretórios.
- `pandas`: Para organização e manipulação dos dados em DataFrames.

In [1]:
import wfdb
from wfdb import rdrecord
import os
import pandas as pd

## 2. Definição dos registros, caminho dos Arquivos, iteração sobre os registros e exibição dos canais disponíveis

O código irá iterar sobre cada um desses registros e verificar os canais disponíveis.


In [ ]:
registros = [
    '100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
    '111', '112', '113', '114', '115', '116', '117', '118', '119',
    '121', '122', '123', '124', '200', '201', '202', '203', '205', '207',
    '208', '209', '210', '212', '213', '214', '215', '217', '219', '220',
    '221', '222', '223', '228', '230', '231', '232', '233', '234'
]

local_path = '/home/joaovfg/PFC-WPW/mit-bih-arrhythmia-database-1.0.0/'

for reg in registros:
    try:
        record = rdrecord(os.path.join(local_path, reg))
        canais = record.sig_name
        print(f'Registro {reg}: Canais disponíveis -> {canais}')
    except Exception as e:
        print(f'Erro ao carregar registro {reg}: {e}')

## 4. Processamento de Registros com Canal MLII

Verifica-se se o canal MLII está presente em cada registro e carrega-se apenas esse canal, caso disponível.

In [13]:
# Dicionário para armazenar registros com canal MLII
registros_com_mlII = {}

# Primeiro, carregamos os registros com canal MLII
for reg in registros:
    try:
        record = rdrecord(os.path.join(local_path, reg), channels=None)
        canais = record.sig_name

        # Verifica se o canal MLII está presente
        if 'MLII' not in canais:
            print(f"Registro {reg} ignorado: canal MLII não encontrado.")
            continue

        idx_mlII = canais.index('MLII')
        record_mlII = rdrecord(os.path.join(local_path, reg), channels=[idx_mlII])

        registros_com_mlII[reg] = record_mlII
        print(f"Registro {reg} carregado com canal MLII (índice {idx_mlII}).")

    except Exception as e:
        print(f"Erro ao processar registro {reg}: {e}")

Registro 100 carregado com canal MLII (índice 0).
Registro 101 carregado com canal MLII (índice 0).
Registro 102 ignorado: canal MLII não encontrado.
Registro 103 carregado com canal MLII (índice 0).
Registro 104 ignorado: canal MLII não encontrado.
Registro 105 carregado com canal MLII (índice 0).
Registro 106 carregado com canal MLII (índice 0).
Registro 107 carregado com canal MLII (índice 0).
Registro 108 carregado com canal MLII (índice 0).
Registro 109 carregado com canal MLII (índice 0).
Registro 111 carregado com canal MLII (índice 0).
Registro 112 carregado com canal MLII (índice 0).
Registro 113 carregado com canal MLII (índice 0).
Registro 114 carregado com canal MLII (índice 1).
Registro 115 carregado com canal MLII (índice 0).
Registro 116 carregado com canal MLII (índice 0).
Registro 117 carregado com canal MLII (índice 0).
Registro 118 carregado com canal MLII (índice 0).
Registro 119 carregado com canal MLII (índice 0).
Registro 121 carregado com canal MLII (índice 0).


## 5. Tratamento de registros com morfologias atípicas e ritmos acelerados

Os registros 108,111 e 124 possuem morfologias atípicas, enquanto os registros 200, 203 e 207 possuem o mesmo com ritmos acelerados, o que atrapalha na segmentação dos sinais.

In [14]:
# Registros a serem excluídos devido a morfologias atípicas e ritmos acelerados
registros_excluir = ['108', '111', '124', '200', '203', '207']

# Lista para armazenar os registros filtrados
registros_filtrados_com_mlII = {}

# Agora, verificamos e removemos os registros a serem excluídos
for reg, record in registros_com_mlII.items():
    if reg in registros_excluir:
        print(f"Registro {reg} excluído devido a morfologias atípicas ou ritmos acelerados.")
        continue  # Exclui o registro

    # Se o registro não for excluído, é armazenado na lista de registros filtrados
    registros_filtrados_com_mlII[reg] = record
    print(f"Registro {reg} mantido após verificação.")


Registro 100 mantido após verificação.
Registro 101 mantido após verificação.
Registro 103 mantido após verificação.
Registro 105 mantido após verificação.
Registro 106 mantido após verificação.
Registro 107 mantido após verificação.
Registro 108 excluído devido a morfologias atípicas ou ritmos acelerados.
Registro 109 mantido após verificação.
Registro 111 excluído devido a morfologias atípicas ou ritmos acelerados.
Registro 112 mantido após verificação.
Registro 113 mantido após verificação.
Registro 114 mantido após verificação.
Registro 115 mantido após verificação.
Registro 116 mantido após verificação.
Registro 117 mantido após verificação.
Registro 118 mantido após verificação.
Registro 119 mantido após verificação.
Registro 121 mantido após verificação.
Registro 122 mantido após verificação.
Registro 123 mantido após verificação.
Registro 124 excluído devido a morfologias atípicas ou ritmos acelerados.
Registro 200 excluído devido a morfologias atípicas ou ritmos acelerados.
Re

## 5. Extração de batimentos e classes

Extraímos os batimentos cardíacos dos registros, classificando-os em duas categorias:
- **0**: Batimentos não relacionados ao WPW.
- **1**: Batimentos relacionados ao WPW.

Essa classificação é feita com base nas anotações dos registros.


In [16]:
nao_wpw_labels = ['N', 'R', 'L', 'f', 'F', '/', 'V', 'A', 'a', 'j']
wpw_label = '*'

batimentos = []

# Inicializa os contadores
contagem_nao_wpw = 0
contagem_wpw = 0

for reg, record in registros_filtrados_com_mlII.items():
    try:
        atr_path = os.path.join(local_path, f"{reg}_modified")
        if not os.path.exists(f"{atr_path}.atr"):
            atr_path = os.path.join(local_path, reg)

        annotation = wfdb.rdann(atr_path, 'atr')

        for idx, symbol in enumerate(annotation.symbol):
            if symbol in nao_wpw_labels:
                batimentos.append({
                    'record': reg,
                    'sample': annotation.sample[idx],
                    'symbol': symbol,
                    'class': 0  # Não WPW
                })
                contagem_nao_wpw += 1  # Incrementa a contagem de batimentos não WPW
            elif symbol == wpw_label:
                batimentos.append({
                    'record': reg,
                    'sample': annotation.sample[idx],
                    'symbol': symbol,
                    'class': 1  # WPW
                })
                contagem_wpw += 1  # Incrementa a contagem de batimentos WPW

        print(f"Registro {reg}: {len(annotation.sample)} anotações lidas, {len([b for b in batimentos if b['record'] == reg])} relevantes.")

    except Exception as e:
        print(f"Erro ao ler anotações do registro {reg}: {e}")

# Exibe o total de amostras relevantes por classe
print(f"\nTotal de amostras relevantes:")
print(f"Classe 0 (Não WPW): {contagem_nao_wpw} batimentos")
print(f"Classe 1 (WPW): {contagem_wpw} batimentos")

# Exibe o total geral
total_batimentos = contagem_nao_wpw + contagem_wpw
print(f"Total de batimentos: {total_batimentos}")


Registro 100: 2274 anotações lidas, 2273 relevantes.
Registro 101: 1874 anotações lidas, 1863 relevantes.
Registro 103: 2091 anotações lidas, 2084 relevantes.
Registro 105: 2691 anotações lidas, 2567 relevantes.
Registro 106: 2098 anotações lidas, 2027 relevantes.
Registro 107: 2140 anotações lidas, 2137 relevantes.
Registro 109: 2535 anotações lidas, 2532 relevantes.
Registro 112: 2550 anotações lidas, 2539 relevantes.
Registro 113: 1796 anotações lidas, 1795 relevantes.
Registro 114: 1890 anotações lidas, 1877 relevantes.
Registro 115: 1962 anotações lidas, 1953 relevantes.
Registro 116: 2421 anotações lidas, 2412 relevantes.
Registro 117: 1539 anotações lidas, 1535 relevantes.
Registro 118: 2301 anotações lidas, 2278 relevantes.
Registro 119: 2094 anotações lidas, 1987 relevantes.
Registro 121: 1876 anotações lidas, 1863 relevantes.
Registro 122: 2479 anotações lidas, 2476 relevantes.
Registro 123: 1519 anotações lidas, 1518 relevantes.
Registro 201: 2039 anotações lidas, 1962 relev

## 6. Criação do DataFrame e Salvamento em CSV

Após extrair os batimentos e suas classes, adiciona-se a coluna "channel" para indicar o canal de ECG utilizado (MLII). Em seguida, cria-se um DataFrame e salva-se os dados em um arquivo CSV.


In [17]:
# Adiciona a coluna 'channel'
for b in batimentos:
    b['channel'] = 'MLII'

# Cria o DataFrame
df_batimentos = pd.DataFrame(batimentos, columns=['record', 'sample', 'symbol', 'channel', 'class'])

# Exibe o DataFrame no console
print(df_batimentos)

      record  sample symbol channel  class
0        100      77      N    MLII      0
1        100     370      N    MLII      0
2        100     662      N    MLII      0
3        100     946      N    MLII      0
4        100    1231      N    MLII      0
...      ...     ...    ...     ...    ...
92042    234  648797      N    MLII      0
92043    234  649040      N    MLII      0
92044    234  649292      N    MLII      0
92045    234  649536      N    MLII      0
92046    234  649772      N    MLII      0

[92047 rows x 5 columns]


In [19]:
# Salva o DataFrame em um arquivo CSV
df_batimentos.to_csv('/home/joaovfg/PFC-WPW/csv/batimentos_filtrados.csv', index=False)
print("Arquivo 'batimentos_filtrados.csv' salvo com sucesso.")

Arquivo 'batimentos_filtrados.csv' salvo com sucesso.
